In [6]:
from Bio.Seq import Seq
from Bio.Alphabet import IUPAC
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.motifs import create, Motif
from Bio.SeqUtils import ProtParam
import numpy as np
import pandas as pd
from scipy.stats import chi2_contingency
import matplotlib.pyplot as plt
import math

In [35]:
with open(r"D:\study\paper\ubiquitination\Arab\dataset\win31\test_pos_win31.txt", 'r') as f1:
    pos_lines = f1.readlines()
    
with open(r"D:\study\paper\ubiquitination\Arab\dataset\win31\test_neg_win31.txt", 'r') as f2:
    neg_lines = f2.readlines()

In [36]:
# 创建自定义字母表
my_alphabet = IUPAC.IUPACProtein.letters + "X"

In [37]:
pos_seqs = create([Seq(line.strip(),alphabet=my_alphabet) for line in pos_lines])
neg_seqs = create([Seq(line.strip(),alphabet=my_alphabet) for line in neg_lines])

In [38]:
pos_seqs_counts = pos_seqs.counts
neg_seqs_counts = neg_seqs.counts

In [39]:
pos_counts_df = pd.DataFrame(pos_seqs_counts)
neg_counts_df = pd.DataFrame(neg_seqs_counts)

In [40]:
chi_square_statistical_difference_table = np.zeros((20,31))

In [41]:
aa = IUPAC.IUPACProtein.letters

In [42]:
def chi2_calculate(chi2_table):
    chi2_value = (chi2_table[-1,-1]**2 / (chi2_table[0,-1]*chi2_table[1,-1])) * (np.sum([chi2_table[0,f]**2/chi2_table[-1,f] for f in range(20)]) - chi2_table[0,-1]/chi2_table[-1,-1])
    return chi2_value

In [43]:
for i in range(31):
    
    if i != 16:
    
        chi2_table = np.zeros((3,21))

        chi2_table[0,:20] = pos_counts_df.iloc[i, :20]
        chi2_table[1,:20] = neg_counts_df.iloc[i, :20] / 3

        chi2_table[0,-1] = np.sum(chi2_table[0,:20])
        chi2_table[1,-1] = np.sum(chi2_table[1,:20])


        chi2_table[2,:] = np.sum(chi2_table[0:2, :],axis=0)
        
        difference_value_list = []
        for a in range(len(aa)):
            chi2_table_pos = chi2_table.copy()
            chi2_table_neg = chi2_table.copy()
            chi2_table_pos[0,a] += 1
            chi2_table_neg[1,a] += 1
            pos_chi2_value = chi2_calculate(chi2_table_pos)
            neg_chi2_value = chi2_calculate(chi2_table_neg)
            difference_value = pos_chi2_value - neg_chi2_value
            difference_value_list.append(difference_value)

        chi_square_statistical_difference_table[:,i] = difference_value_list
        
    else:
        chi_square_statistical_difference_table[:,i] = np.nan

In [44]:
x_mean = np.mean(chi_square_statistical_difference_table,axis=0)

In [45]:
chi_square_statistical_difference_table_with_x = np.vstack((chi_square_statistical_difference_table,x_mean))

In [46]:
pos_feature = np.zeros((len(pos_lines), 31))
for i in range(len(pos_lines)):
    pos = pos_lines[i].strip()
    j = 0
    for p in pos:
        p_index = my_alphabet.index(p)
        feature_value = chi_square_statistical_difference_table_with_x[p_index,j]
        pos_feature[i,j] = feature_value
        j += 1

In [47]:
pos_feature

array([[3.19856244, 3.66946914, 3.82484763, ..., 4.60185455, 3.65365843,
        4.15796699],
       [3.19856244, 4.11503087, 2.81674825, ..., 3.82831446, 3.39268283,
        3.57233784],
       [3.08143534, 4.17340011, 3.82484763, ..., 3.7278181 , 4.79040793,
        4.48897478],
       ...,
       [3.93034289, 3.87435424, 3.91292867, ..., 3.79373667, 2.40097554,
        4.38111668],
       [4.61911413, 3.90298081, 3.76854421, ..., 4.60185455, 3.77739687,
        4.38111668],
       [4.61911413, 3.98263348, 5.00846297, ..., 4.535161  , 4.2776001 ,
        4.38111668]])

In [48]:
neg_feature = np.zeros((len(neg_lines), 31))
for i in range(len(neg_lines)):
    neg = neg_lines[i].strip()
    j = 0
    for n in neg:
        n_index = my_alphabet.index(n)
        feature_value = chi_square_statistical_difference_table_with_x[n_index,j]
        neg_feature[i,j] = feature_value
        j += 1

In [49]:
feature = np.vstack((pos_feature,neg_feature))

In [50]:
label = np.vstack((np.ones((len(pos_lines),1)), np.zeros((len(neg_lines),1))))

In [51]:
data = np.hstack((label, feature))

In [52]:
data

array([[1.        , 3.19856244, 3.66946914, ..., 4.60185455, 3.65365843,
        4.15796699],
       [1.        , 3.19856244, 4.11503087, ..., 3.82831446, 3.39268283,
        3.57233784],
       [1.        , 3.08143534, 4.17340011, ..., 3.7278181 , 4.79040793,
        4.48897478],
       ...,
       [0.        , 4.15900003, 3.98056816, ..., 3.50541268, 4.44016025,
        3.53099931],
       [0.        , 3.00242418, 3.90298081, ..., 3.55406387, 4.35998046,
        3.71075928],
       [0.        , 4.26551297, 3.90298081, ..., 3.98119626, 3.66382632,
        4.74632   ]])

In [53]:
new_data = np.hstack((data[:,:17], data[:,18:]))

In [54]:
col_name = ['CSDT_'+str(i+1) for i in range(30)]
col_name.insert(0,'label')

In [55]:
df = pd.DataFrame(new_data,columns=col_name)

In [56]:
df.to_csv(r"D:\study\paper\ubiquitination\Arab\feature_extraction\win31_new\test\test_csdt.csv", index=False)